## Imports

In [3]:
# Imports
import requests
import re
from bs4 import BeautifulSoup

## Global Values

In [13]:
parent_folder = ""
html_folder = "full_html"
text_folder = "plain_text"

## Misc. methods

In [22]:
# Filters the content out of a minutes-page (FOMC)
def filter_text(url, save = False):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    content = soup.find(id = "article")
    if(content is not None):
        text = content.get_text()
    else:
        content = soup.find(id = "leftText")
        if content is not Note:
            text = content.get_text()
        else:
            tables = soup.find_all("table")
            text = ""
            for table in tables:
                text += table.get_text()
        
    if save == True:
        filename_html = "".join(re.findall('\d+', url)) + "_fomc_html"
        filename_text = "".join(re.findall('\d+', url)) + "_fomc_text"
        save_content(content = soup, folder = parent_folder + html_folder, filename = filename_html)
        save_content(content = text, folder = parent_folder + text_folder, filename = filename_text)
    return text

In [17]:
# Save soup-object in folder
def save_content(content, folder, filename):
    with open(folder + "/" + filename + ".txt", "w") as file:
        file.write(content.encode("utf-8"))

## Retrieving content from current FOMC Meetings page (2013 - today)

In [7]:
# Get page content of current meetings page
current_fomc_page = requests.get("https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm")
current_fomc_soup = BeautifulSoup(current_fomc_page.content, 'html.parser')
current_fomc_blocks = current_fomc_soup.select(".fomc-meeting__minutes")

In [18]:
# Get URL, Date and Minutes of each Meeting
# Structure of each tuple in the list: (date, url, content) 
# date format: yyyymmdd
current_minutes_fomc = []
for m in current_fomc_blocks:
    for a in m.select("a"):
        if(".htm" in a["href"]):
            url = "https://www.federalreserve.gov" + a["href"]
            date = "".join(re.findall('\d+', url))
            article = filter_text(url, True)
            current_minutes_fomc.append((date, url, article))

## Retrieving content from current FOMC Meetings page (2012 and older)

In [19]:
# Get page content of historical meetings page
historical_fomc_page = requests.get("https://www.federalreserve.gov/monetarypolicy/fomc_historical_year.htm")
historical_fomc_soup = BeautifulSoup(historical_fomc_page.content, 'html.parser')
# historical_archive_minutes_pages = historical_archive_soup.select(".fomc-meeting__minutes")

In [20]:
# Get url for each historical year
blocks = historical_fomc_soup.select(".panel-default")
historical_minutes_urls = []
for block in blocks:
    for a in block.select("a"):
        historical_minutes_urls.append("https://www.federalreserve.gov" + a["href"])

In [21]:
# Get URL, Date and Minutes of each Meeting
# Structure of each tuple in the list: (date, url, content) 
# date format: yyyymmdd
historical_minutes_fomc = []
for url in historical_minutes_urls:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    blocks = soup.select(".panel-default")
    for block in blocks:
        for a in block.select("a"):
            # urls are not consistent over the years
            if ("minutes" in a["href"]) and (".htm" in a["href"]) and ("#" not in a["href"]):
                if("https://www.federalreserve.gov" in a["href"]):
                    url = a["href"]
                else:
                    url= "https://www.federalreserve.gov" + a["href"]
                # date is extracted from the url
                date = "".join(re.findall('\d+', url))
                article = filter_text(url, True)
                historical_minutes_fomc.append((date, url, article))

AttributeError: 'NoneType' object has no attribute 'get_text'

## Unite collections

In [67]:
minutes_fomc = sorted(historical_minutes_fomc) + sorted(current_minutes_fomc)

# Testing Area

In [10]:
with open("full_html/20180131_fomc_html.txt", "w") as f: 
    f.write("Hello World") 